In [29]:
import h5py
import numpy as np
import fitsio

In [76]:
covmat = fitsio.read("2pt_NG_final_2ptunblind_02_26_21_wnz_maglim_covupdate.fits", ext="COVMAT")
covmat = covmat[:400, :400]

xip = fitsio.read("2pt_NG_final_2ptunblind_02_26_21_wnz_maglim_covupdate.fits", ext="xip")
xim = fitsio.read("2pt_NG_final_2ptunblind_02_26_21_wnz_maglim_covupdate.fits", ext="xim")

def extract_cov(covmat, msk):
    dim = np.sum(msk)
    ncov = np.zeros((dim, dim), dtype="f8")
    for ni, i in enumerate(np.where(msk)[0]):
        for nj, j in enumerate(np.where(msk)[0]):
            ncov[ni, nj] = covmat[i, j]
    return ncov


wgts = []
for i in [1, 2, 3, 4]:
    msk = np.concatenate([
        (xip["BIN1"] == i) & (xip["BIN2"] == i),
        (xim["BIN1"] == i) & (xim["BIN2"] == i),
    ], axis=0)

    inv_covmat = np.linalg.inv(extract_cov(covmat, msk))

    dv = np.concatenate([
        xip[(xip["BIN1"] == i) & (xip["BIN2"] == i)]["VALUE"],
        xim[(xim["BIN1"] == i) & (xim["BIN2"] == i)]["VALUE"],
    ], axis=0)
    
    wgts.append(np.dot(dv, np.dot(inv_covmat, dv)))
    
wgts = np.array(wgts)
wgts /= np.sum(wgts)
print(wgts)

[0.0423873  0.11593369 0.38744974 0.45422926]


In [2]:
f = h5py.File('/project/projectdirs/des/www/y3_cats/Y3_mastercat___UNBLIND___final_v1.0_DO_NOT_USE.h5', mode='r')

In [80]:
amag_g = f["catalog/gold/mag_auto_g"][:]
amag_i = f["catalog/gold/mag_auto_i"][:]
mag_g = f["catalog/gold/sof_cm_mag_corrected_g"][:]
mag_i = f["catalog/gold/sof_cm_mag_corrected_i"][:]
gmi = mag_g - mag_i
agmi = amag_g - amag_i

mag_z = f["catalog/gold/sof_cm_mag_corrected_z"][:]
imz = mag_i - mag_z

In [83]:
vals = []
nums = []
iz_vals = []
for t in ['','_bin1','_bin2','_bin3','_bin4']:
    mask = f['index/select' + t][:]
    print(t + ":" if t != '' else '  all' + ":", np.mean(gmi[mask]), np.median(gmi[mask]), np.mean(agmi[mask]), np.median(agmi[mask]))
    if t != "":
        vals.append(np.mean(gmi[mask]))
        nums.append(np.sum(mask))
        iz_vals.append(np.mean(imz[mask]))

vals = np.array(vals)
nums = np.array(nums)
iz_vals = np.array(iz_vals)

  all: 1.4142729295407583 1.2464192139299684 1.984526571381404 1.3205280303955078
_bin1: 0.9909833938539456 0.9506278593561817 1.0467711622347917 1.0131816864013672
_bin2: 1.3773552537688294 1.2992486401485372 1.444433394599058 1.3850364685058594
_bin3: 1.720342822816316 1.531531922184758 2.197417569699361 1.6174983978271484
_bin4: 1.5685767511916027 1.2908023827086943 3.2496097392493795 1.37750244140625


In [84]:
c = np.sum(vals * nums * wgts) / np.sum(nums * wgts), np.sum(vals * nums) / np.sum(nums)
c, c[0] - c[1]

((1.5800720362863738, 1.4143438777333979), 0.16572815855297596)

In [85]:
c = np.sum(iz_vals * nums * wgts) / np.sum(nums * wgts), np.sum(iz_vals * nums) / np.sum(nums)
c, c[0] - c[1]

((0.36838359619743444, 0.2887834729007413), 0.07960012329669314)